# Overview

In today's session, groups of students will work on a few sample exercises to get familiarized with Ethereum transaction data.

1. Form groups of 3-4
2. Groups will work on Ethereum transactions between block 10230000 to 10240000.
3. Every group starts with general data preparation and then selects a focus area : Sender analytics, Receiver analytics, Fee analytics, Block analytics
4. We will reserve last 10 minutes for a short 2-3 minute presentation from each group


# Data Prep

1. Load csv files to a dataframe or any other data structure of your choice
2. Join individual dataframes to a single dataframe
3. Data checks to ensure that the data is read correctly

In [14]:
# import packages
import pandas as pd
import os
import matplotlib.pyplot as plt
%matplotlib inline

In [15]:
# read files to individual dataframes and convert to a single dataframe
# remove duplicate transactions

In [16]:
path = "./data"
dfs = []
for f in os.listdir(path):
    dfs.append(pd.read_csv(os.path.join(path, f)))
df = pd.concat(dfs, axis=0, ignore_index=True)
# Remove duplicate transactions
df.drop_duplicates("Tx_Hash", inplace=True)

In [17]:
# 1. What is the shape (number of rows and number of columns) of the combined dataframe

In [19]:
# 2. What is the minimum and maximum blocknumber

In [21]:
# 3. Is Tx_Hash column a primary key in this table ?

# Focus area : Sender analytics
Focus on the sender field (Tx_From) of each transaction and answer the following questions

In [23]:
# 1. For each unique address, calculate the number of times the address has initiated a transaction

In [25]:
# 2. Find the address with the most number of transactions

In [27]:
# 3a. What fraction of addresses have sent a single transaction


In [29]:
# 3b. What fraction of addresses have sent two transactions


In [31]:
# 4. Draw a graph with number of transactions sent by address on X-axis and Number of addresses on Y-axis

In [33]:
# 5. Can you think of any other sender metrics


# Focus area : Receiver analytics
Focus on the sender field (Tx_To) of each transaction and answer the following questions

In [36]:
# 1. For each unique address, calculate the ￼number of times the address has received a transaction

In [38]:
# 2. Ethereum contract deployment transactions have an empty To value. How many contracts are deployed in this dataset ?

In [40]:
# 3a. What fraction of addresses have received a single transaction


In [42]:
# 3b. What fraction of addresses have received two transactions df.func_hash.value_counts().idxmax()

In [44]:
# 4. Draw a graph with number of transactions sent to address on X-axis and Number of addresses on Y-axis

In [47]:
# 5. Can you think of any other receiver metrics


# Focus area : Fee analytics

Focus on  'Tx_Gas','Tx_GasPrice', 'Tx_Value', 'Tx_Nonce', 'Tx_Cost' fields of the dataframe

In [49]:
# 1. Using the Tx_Value field, find the transaction (hash) with highest value

In [51]:
# 2. Using etherscan.io and coinmarketcap.com, find the dollar amount of the above transaction then and today

In [53]:
# 3. Find the most common Gas cost ? What kind of transaction does it represent ?

In [55]:
# 4. Find the transaction with highest fee and what is it's cost today ?

In [57]:
# 5. Do you think the above transaction is an outlier ? Can you come up with a theory to explain this transaction ?

In [58]:
# 6. Find the block with highest average gas price. Does it have the above transaction ?

# Focus area : Block analytics

Group by "Block_Number" to get block level stats

In [59]:
#1. Find the block with highest number of transactions and how many transactions does it have

In [61]:
#2. Find the block with least number of transactions and how many transactions does it have

In [63]:
# 3. Using etherscan.io, find the average block time during this period

In [65]:
# 4. The first 8 characters (if its exists) of Tx_Data is the keccak256 hash of the function that is invoked in the transaction. Use https://www.4byte.directory/signatures/ to find a signature of the most common function during this time.